In [2]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [3]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2026-01-07 11:37:18.668 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\denni\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 5081.67it/s]


In [4]:
len(train), len(valid)

(81, 20)

In [5]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 30, 3]),
 tensor([18, 10, 17, 10,  8, 15, 17,  0,  0,  0,  2,  1, 14, 16, 10,  2,  4, 11,
         16, 15,  9,  7,  8, 18, 18, 16, 12,  7, 18,  1, 11,  5]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [6]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [7]:
model = rnn_models.BaseRNN(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    horizon=20,
)

Test the model. What is the output shape you need? Remember, we are doing classification!

In [8]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [9]:
accuracy(y, yhat)

0.03125

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [10]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [11]:
yhat[0]

tensor([-0.0205, -0.0310,  0.0290,  0.1616, -0.0082,  0.0675, -0.0680, -0.0803,
         0.0125,  0.0340, -0.0595,  0.0148,  0.1036, -0.0333,  0.1702, -0.0325,
        -0.0459,  0.1505,  0.0730, -0.1206], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [12]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(2.9868, grad_fn=<NllLossBackward0>)

In [13]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [14]:
settings = TrainerSettings(
    epochs=50, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 50
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [20]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [52]:
# LSTM variant: swap GRU for LSTM and use the last hidden state
class LSTMmodel(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.LSTM(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        out, (h_n, c_n) = self.rnn(x)
        # h_n shape: (num_layers, batch, hidden_size) -> take last layer
        last = h_n[-1]
        yhat = self.linear(last)
        return yhat

# Quick instantiation example (use later when training):
# config_lstm = ModelConfig(input_size=3, hidden_size=128, num_layers=2, output_size=20, dropout=0.5)
# model = LSTMmodel(config_lstm)
# yhat = model(x); yhat.shape

In [62]:
# Training snippet for LSTMmodel (mlflow + Trainer)
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

# configure LSTM and train
with mlflow.start_run():
    mlflow.set_tag("model", "256-hidden-LSTM 2-layer 0.3-dropout")
    mlflow.set_tag("dev", "Dennis")
    config_lstm = ModelConfig(
        input_size=3,
        hidden_size=256,
        num_layers=2,
        output_size=20,
        dropout=0.3,
    )

    model = LSTMmodel(config_lstm)

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "lstm-model.pt")
        torch.save(model, modelpath)

2026-01-07 13:11:30.059 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20260107-131130
2026-01-07 13:11:30.060 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 44.09it/s]
2026-01-07 13:11:32.070 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5014 test 2.3797 metric ['0.2016']
100%|██████████| 81/81 [00:02<00:00, 38.89it/s]
2026-01-07 13:11:34.339 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.0589 test 2.0007 metric ['0.3047']
100%|██████████| 81/81 [00:01<00:00, 45.12it/s]
2026-01-07 13:11:36.303 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.4716 test 1.3311 metric ['0.4562']
100%|██████████| 81/81 [00:01<00:00, 42.98it/s]
2026-01-07 13:11:38.359 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.1943 test 1.1116 metric ['0.5344']
100%|██████████| 81/81 [00:01<00:00, 47.02it/s]
2026-01-07 13:11:40.25

🏃 View run mercurial-sloth-140 at: http://127.0.0.1:5000/#/experiments/2/runs/3cf258aeca5f4704904aa1f4bd9c9a04
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [ ]:
mlflow.end_run()

In [68]:
# Conv1D + GRU variant: apply temporal convolution before the RNN (PyTorch expects Conv1d input as (B, C, T))
class ConvGRUmodel(nn.Module):
    def __init__(self, config, conv_channels=32, kernel_size=5) -> None:
        super().__init__()
        self.config = config
        self.conv = nn.Conv1d(in_channels=config.input_size, out_channels=conv_channels, kernel_size=kernel_size, padding=(kernel_size-1)//2)
        self.bn = nn.BatchNorm1d(conv_channels)
        self.relu = nn.ReLU()
        self.rnn = nn.GRU(input_size=conv_channels, hidden_size=config.hidden_size, num_layers=config.num_layers, batch_first=True, dropout=config.dropout)
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        # x: (batch, timesteps, channels) -> permute to (batch, channels, timesteps) for Conv1d
        x = x.permute(0, 2, 1)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        # back to (batch, timesteps, channels=conv_channels)
        x = x.permute(0, 2, 1)
        out, h = self.rnn(x)
        last = h[-1]
        yhat = self.linear(last)
        return yhat

# Training snippet for ConvGRU (use the best-performing hyperparams as baseline)
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "ConvGRU-32conv-256hid-2layer-0.0dropout-5-kernel")
    mlflow.set_tag("dev", "Dennis")
    config_conv = ModelConfig(
        input_size=3,
        hidden_size=256,
        num_layers=2,
        output_size=20,
        dropout=0.0,
    )

    model = ConvGRUmodel(config_conv, conv_channels=32, kernel_size=5)

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "convgru-model.pt")
        torch.save(model, modelpath)

2026-01-07 13:39:19.581 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20260107-133919
2026-01-07 13:39:19.582 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:03<00:00, 26.38it/s]
2026-01-07 13:39:22.901 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5944 test 2.4013 metric ['0.1984']
100%|██████████| 81/81 [00:02<00:00, 31.63it/s]
2026-01-07 13:39:25.720 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.7826 test 2.0090 metric ['0.3047']
100%|██████████| 81/81 [00:03<00:00, 26.93it/s]
2026-01-07 13:39:29.013 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.0084 test 0.8641 metric ['0.6281']
100%|██████████| 81/81 [00:02<00:00, 31.79it/s]
2026-01-07 13:39:31.833 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.6364 test 0.4904 metric ['0.8375']
100%|██████████| 81/81 [00:02<00:00, 29.96it/s]
2026-01-07 13:39:34.78

🏃 View run bustling-calf-417 at: http://127.0.0.1:5000/#/experiments/2/runs/00b8e25db2854afa8fa5f5707dcaa889
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [21]:
config = ModelConfig(
    input_size=3,
    hidden_size=128,
    num_layers=2,
    output_size=20,
    dropout=0.5,
)


In [51]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "512-hidden-GRU 1-layer 0.3-dropout")
    mlflow.set_tag("dev", "Dennis")
    config = ModelConfig(
        input_size=3,
        hidden_size=512,
        num_layers=1,
        output_size=20,
        dropout=0.3,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2026-01-07 12:16:11.373 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20260107-121611
2026-01-07 12:16:11.374 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:03<00:00, 25.58it/s]
2026-01-07 12:16:14.831 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4408 test 2.1745 metric ['0.2953']
100%|██████████| 81/81 [00:02<00:00, 29.71it/s]
2026-01-07 12:16:17.817 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.5785 test 1.4091 metric ['0.4781']
100%|██████████| 81/81 [00:02<00:00, 29.96it/s]
2026-01-07 12:16:20.778 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.7509 test 0.5085 metric ['0.8359']
100%|██████████| 81/81 [00:02<00:00, 29.80it/s]
2026-01-07 12:16:23.744 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3656 test 0.2600 metric ['0.9359']
100%|██████████| 81/81 [00:02<00:00, 30.81it/s]
2026-01-07 12:16:26.61

🏃 View run abundant-loon-913 at: http://127.0.0.1:5000/#/experiments/2/runs/8e42aaca0d2e474c8e55c8ea80ad0087
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


Try to update the code above by changing the hyperparameters.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [32]:
trainer.loop() # if you want to pick up training, loop will continue from the last epoch

100%|██████████| 81/81 [00:01<00:00, 45.23it/s]
2026-01-07 11:57:16.984 | INFO     | mltrainer.trainer:report:175 - Resuming epochs from previous training at 63
2026-01-07 11:57:17.107 | INFO     | mltrainer.trainer:report:209 - Epoch 63 train 0.0600 test 0.1814 metric ['0.9578']
2026-01-07 11:57:17.109 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.1803, current loss 0.1814.Counter 6/5.
2026-01-07 11:57:17.109 | INFO     | mltrainer.trainer:loop:103 - Interrupting loop due to early stopping patience.
2026-01-07 11:57:17.110 | INFO     | mltrainer.trainer:loop:108 - early_stopping_save was false, using latest model.Set to true to retrieve best model.
  0%|          | 0/50 [00:02<?, ?it/s]


In [45]:
mlflow.end_run()

Excercises:

- try to improve the RNN model
- test different things. What works? What does not?
- experiment with either GRU or LSTM layers, create your own models. Have a look at `mltrainer.rnn_models` for inspiration. 
- experiment with adding Conv1D layers. Think about the necessary input-output dimensions of your tensors before and after each layer.

You should be able to get above 90% accuracy with the dataset.
Create a report of 1 a4 about your experiments.